In [7]:
# -*- coding: utf-8 -*-
"""
시간대별 AMT/CNT 예측 (수원시 한식 동별 데이터)
- 입력: 날짜(TA_YMD), DONG, (기상청 API로 가져온) 시간대별 TEMP/RAIN
- 출력: 시간대별 AMT, CNT

모델 구성:
- HOUR=1  : "Deep" => MLPRegressor(신경망)
- HOUR=2~10: XGB   => xgboost.XGBRegressor
"""

import os
import math
import joblib
import requests
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, timezone

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error

from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from dotenv import load_dotenv

# =========================
# A) 설정
# =========================
load_dotenv('./../01_python/.env')
SERVICE_KEY = os.getenv('RAIN_ID')
DATA_CSV = r"C:/data/수원시 한식 동별 데이터백업.csv"
MODEL_DIR = "./models_hourly_amt_cnt"
os.makedirs(MODEL_DIR, exist_ok=True)

# ---- 기상청 단기예보(getVilageFcst) 엔드포인트 ----
# 공공데이터포털 / 기상청 단기예보(구 동네예보) 서비스
KMA_VILAGE_URL = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"

# base_time 발표시각(단기예보): 02,05,08,11,14,17,20,23시 (일 8회)
BASE_TIMES = ["2300", "2000", "1700", "1400", "1100", "0800", "0500", "0200"]

# HOUR(01~10) 구간(너가 준 항목요약 기준)
# 01: 00:00~06:59
# 02: 07:00~08:59
# 03: 09:00~10:59
# 04: 11:00~12:59
# 05: 13:00~14:59
# 06: 15:00~16:59
# 07: 17:00~18:59
# 08: 19:00~20:59
# 09: 21:00~22:59
# 10: 23:00~23:59
HOUR_SLOT_TO_REP_TIME = {
    1: "0300",  # 새벽 구간 대표 시각(중간값 근사)
    2: "0800",
    3: "1000",
    4: "1200",
    5: "1400",
    6: "1600",
    7: "1800",
    8: "2000",
    9: "2200",
    10: "2300",
}

# ---- DONG -> (nx, ny) 격자 좌표 매핑 (필수) ----
# ※ 기상청 단기예보는 nx, ny(격자좌표)가 필요합니다.
# 여기엔 예시로 비워두고, 너는 '동별 nx/ny' 파일을 만들어 로딩하거나 아래 dict를 채워야 합니다.
# 추천: dong_grid.csv 컬럼 예) DONG,nx,ny 로 만들어 로딩
DONG_GRID_CSV = "C:/data/dong_grid.csv"


# =========================
# B) 유틸: 요일 계산 (1~7, 월=1 ... 일=7)
# =========================
def ymd_to_day_1to7(yyyymmdd: str) -> int:
    dt = datetime.strptime(str(yyyymmdd), "%Y%m%d")
    # weekday(): 월0~일6 -> +1 해서 월1~일7
    return dt.weekday() + 1


# =========================
# C) 기상청 API: base_date/base_time 결정
# =========================
def pick_base_datetime_kst(now_kst: datetime) -> tuple[str, str]:
    """
    현재(KST) 기준으로 가장 최근 발표시각(base_time)을 선택.
    (예: 14:30이면 base_time=1400, base_date=오늘)
    (예: 01:10이면 base_time=2300, base_date=어제)
    """
    today = now_kst.strftime("%Y%m%d")
    hhmm = now_kst.strftime("%H%M")

    for bt in BASE_TIMES:
        if hhmm >= bt:
            return today, bt

    # 새벽 00:00~01:59 등은 전날 23:00 발표 사용
    yday = (now_kst - timedelta(days=1)).strftime("%Y%m%d")
    return yday, "2300"


# =========================
# D) 기상청 단기예보 호출 + 특정 날짜/시각 TMP, PCP 추출
# =========================
def fetch_vilage_fcst_items(service_key: str, base_date: str, base_time: str, nx: int, ny: int,
                            num_of_rows: int = 2000) -> list[dict]:
    params = {
        "serviceKey": service_key,
        "pageNo": 1,
        "numOfRows": num_of_rows,
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": nx,
        "ny": ny,
    }
    r = requests.get(KMA_VILAGE_URL, params=params, timeout=20)
    r.raise_for_status()
    data = r.json()
    items = data["response"]["body"]["items"]["item"]
    return items


def parse_pcp_to_float(pcp_val) -> float:
    """
    단기예보 PCP(1시간 강수량)는 문자열로 '강수없음' 등이 올 수 있음.
    숫자로 변환 실패 시 0 처리.
    """
    if pcp_val is None:
        return 0.0
    s = str(pcp_val).strip()
    if s in ("강수없음", "없음", ""):
        return 0.0
    # '1mm 미만' 같은 케이스도 방어
    s = s.replace("mm", "").replace("미만", "").strip()
    try:
        return float(s)
    except:
        return 0.0


def get_temp_rain_for_datetime(service_key: str, dong: str, target_yyyymmdd: str, target_hhmm: str) -> tuple[float, float]:
    """
    dong, 날짜(YYYYMMDD), 시각(HHMM)에 해당하는 TMP(기온), PCP(1시간강수)를 반환.
    """
    # dong -> nx,ny 로딩
    if not DONG_TO_GRID:
        if os.path.exists(DONG_GRID_CSV):
            g = pd.read_csv(DONG_GRID_CSV)
            for _, row in g.iterrows():
                DONG_TO_GRID[str(row["DONG"])] = (int(row["nx"]), int(row["ny"]))
        else:
            raise ValueError(
                f"[필수] DONG_TO_GRID가 비어있습니다. {DONG_GRID_CSV}를 만들거나 DONG_TO_GRID dict를 채워주세요."
            )

    if dong not in DONG_TO_GRID:
        raise ValueError(f"DONG_TO_GRID에 '{dong}'의 nx/ny가 없습니다. dong_grid.csv 또는 dict에 추가하세요.")

    nx, ny = DONG_TO_GRID[dong]

    # base_date/time는 '현재 시각' 기준 최근 발표본 사용
    # (예측 날짜가 미래여도 단기예보 제공 범위(통상 3일 내)에 있으면 조회 가능)
    now_kst = datetime.now(timezone(timedelta(hours=9)))
    base_date, base_time = pick_base_datetime_kst(now_kst)

    items = fetch_vilage_fcst_items(service_key, base_date, base_time, nx, ny)

    # 원하는 fcstDate, fcstTime + category(TMP/PCP)만 추출
    tmp = None
    pcp = None
    for it in items:
        if it.get("fcstDate") == target_yyyymmdd and it.get("fcstTime") == target_hhmm:
            cat = it.get("category")
            if cat == "TMP":
                tmp = float(it.get("fcstValue"))
            elif cat == "PCP":
                pcp = parse_pcp_to_float(it.get("fcstValue"))

    # 못 찾으면 결측 방어
    if tmp is None:
        tmp = 0.0
    if pcp is None:
        pcp = 0.0

    return tmp, pcp


def get_hourly_weather_features(service_key: str, dong: str, yyyymmdd: str) -> pd.DataFrame:
    """
    날짜+동 기준으로 HOUR=1~10 각각의 대표시각에서 TEMP/RAIN을 가져와
    예측 입력용 DataFrame 생성
    """
    day_1to7 = ymd_to_day_1to7(yyyymmdd)

    rows = []
    for hour_slot in range(1, 11):
        rep_time = HOUR_SLOT_TO_REP_TIME[hour_slot]
        temp, rain = get_temp_rain_for_datetime(service_key, dong, yyyymmdd, rep_time)
        rows.append({
            "TA_YMD": int(yyyymmdd),
            "DONG": dong,
            "HOUR": hour_slot,
            "DAY": day_1to7,
            "TEMP": float(temp),
            "RAIN": float(rain),
        })
    return pd.DataFrame(rows)


# =========================
# E) 모델 학습 (시간대별 저장)
# =========================
def build_preprocess():
    cat_cols = ["DONG"]
    num_cols = ["DAY", "TEMP", "RAIN"]  # 필요 최소 입력
    pre = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
            ("num", "passthrough", num_cols),
        ],
        remainder="drop",
    )
    return pre


def train_and_save_models(random_state: int = 42):
    df = pd.read_csv(DATA_CSV)

    # 타입 정리
    df["DONG"] = df["DONG"].astype(str)
    df["HOUR"] = df["HOUR"].astype(int)
    df["DAY"] = df["DAY"].astype(int)
    df["TEMP"] = df["TEMP"].astype(float)
    df["RAIN"] = df["RAIN"].astype(float)

    # 타겟(AMT, CNT)
    y_cols = ["AMT", "CNT"]

    metrics = []

    for h in range(1, 11):
        sub = df[df["HOUR"] == h].copy()
        sub = sub.dropna(subset=["DONG", "DAY", "TEMP", "RAIN", "AMT", "CNT"])

        X = sub[["DONG", "DAY", "TEMP", "RAIN"]]
        y = sub[y_cols]

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=random_state, shuffle=True
        )

        pre = build_preprocess()

        if h == 1:
            # "Deep": MLPRegressor (torch/tensorflow 없이 신경망)
            base_model = MLPRegressor(
                hidden_layer_sizes=(128, 64),
                activation="relu",
                solver="adam",
                alpha=1e-4,
                batch_size=256,
                learning_rate_init=1e-3,
                max_iter=300,          # ✅ 50 -> 300
                random_state=42,
                early_stopping=True,
                n_iter_no_change=10,   # ✅ 5 -> 10
                verbose=False,
            )
        else:
            # XGB
            base_model = XGBRegressor(
                n_estimators=600,
                learning_rate=0.05,
                max_depth=6,
                subsample=0.9,
                colsample_bytree=0.9,
                reg_alpha=0.0,
                reg_lambda=1.0,
                objective="reg:squarederror",
                random_state=random_state,
                tree_method="hist",
                n_jobs=-1,
            )

        model = MultiOutputRegressor(base_model)

        pipe = Pipeline([
            ("pre", pre),
            ("model", model),
        ])

        pipe.fit(X_train, y_train)

        pred = pipe.predict(X_test)
        mae_amt = mean_absolute_error(y_test["AMT"].values, pred[:, 0])
        mae_cnt = mean_absolute_error(y_test["CNT"].values, pred[:, 1])

        model_path = os.path.join(MODEL_DIR, f"hour_{h:02d}_amt_cnt.joblib")
        joblib.dump(pipe, model_path)

        metrics.append({
            "HOUR": h,
            "MAE_AMT": mae_amt,
            "MAE_CNT": mae_cnt,
            "model_path": model_path,
            "model_type": "MLP(Deep)" if h == 1 else "XGB",
            "n_train": len(X_train),
            "n_test": len(X_test),
        })

        print(f"[HOUR {h:02d}] saved -> {model_path} | MAE(AMT)={mae_amt:,.1f} MAE(CNT)={mae_cnt:,.2f}")

    metrics_df = pd.DataFrame(metrics).sort_values("HOUR")
    metrics_df.to_csv(os.path.join(MODEL_DIR, "metrics_summary.csv"), index=False, encoding="utf-8-sig")
    print("\n=== metrics_summary.csv 저장 완료 ===")
    return metrics_df


# =========================
# F) 예측 (날짜+동 입력 -> 시간대별 AMT/CNT)
# =========================
def load_models():
    models = {}
    for h in range(1, 11):
        p = os.path.join(MODEL_DIR, f"hour_{h:02d}_amt_cnt.joblib")
        if not os.path.exists(p):
            raise FileNotFoundError(f"모델 파일이 없습니다: {p} (먼저 train_and_save_models() 실행)")
        models[h] = joblib.load(p)
    return models


def predict_day(yyyymmdd: str, dong: str) -> pd.DataFrame:
    # 기상청에서 시간대별 TEMP/RAIN 구성

    feat_df = get_hourly_weather_features(SERVICE_KEY, dong, yyyymmdd)

    models = load_models()

    outs = []
    for h in range(1, 11):
        row = feat_df[feat_df["HOUR"] == h][["DONG", "DAY", "TEMP", "RAIN"]]
        pred_amt, pred_cnt = models[h].predict(row)[0]
        outs.append({
            "TA_YMD": yyyymmdd,
            "DONG": dong,
            "HOUR": h,
            "TEMP": float(feat_df.loc[feat_df["HOUR"] == h, "TEMP"].iloc[0]),
            "RAIN": float(feat_df.loc[feat_df["HOUR"] == h, "RAIN"].iloc[0]),
            "PRED_AMT": float(pred_amt),
            "PRED_CNT": float(pred_cnt),
        })

    return pd.DataFrame(outs)


# =========================
# G) 실행 예시
# =========================
if __name__ == "__main__":
    # 1) 학습 + 저장
    metrics_df = train_and_save_models()
    print(metrics_df)

    # 2) 예측 예시
    if not SERVICE_KEY:
        print("\n[경고] 환경변수 KMA_SERVICE_KEY가 비었습니다. 예측을 하려면 SERVICE_KEY를 넣으세요.\n")
    else:
        # dong_grid.csv 또는 DONG_TO_GRID에 해당 동 nx/ny가 있어야 합니다.
        yyyymmdd = "20251230"
        dong = "매교동"
        pred_df = predict_day(yyyymmdd, dong)
        print(pred_df)
        pred_df.to_csv(f"pred_{dong}_{yyyymmdd}.csv", index=False, encoding="utf-8-sig")
        print(f"\n예측 결과 저장: pred_{dong}_{yyyymmdd}.csv")


C:\Users\Admin\anaconda3\envs\ml-dl-nlp\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


[HOUR 01] saved -> ./models_hourly_amt_cnt\hour_01_amt_cnt.joblib | MAE(AMT)=1,192,919.3 MAE(CNT)=17.86
[HOUR 02] saved -> ./models_hourly_amt_cnt\hour_02_amt_cnt.joblib | MAE(AMT)=234,625.9 MAE(CNT)=7.64
[HOUR 03] saved -> ./models_hourly_amt_cnt\hour_03_amt_cnt.joblib | MAE(AMT)=610,639.0 MAE(CNT)=15.00
[HOUR 04] saved -> ./models_hourly_amt_cnt\hour_04_amt_cnt.joblib | MAE(AMT)=1,758,911.7 MAE(CNT)=45.61
[HOUR 05] saved -> ./models_hourly_amt_cnt\hour_05_amt_cnt.joblib | MAE(AMT)=2,206,063.3 MAE(CNT)=43.98
[HOUR 06] saved -> ./models_hourly_amt_cnt\hour_06_amt_cnt.joblib | MAE(AMT)=1,995,921.3 MAE(CNT)=39.93
[HOUR 07] saved -> ./models_hourly_amt_cnt\hour_07_amt_cnt.joblib | MAE(AMT)=1,978,358.2 MAE(CNT)=40.94
[HOUR 08] saved -> ./models_hourly_amt_cnt\hour_08_amt_cnt.joblib | MAE(AMT)=2,354,001.3 MAE(CNT)=40.83
[HOUR 09] saved -> ./models_hourly_amt_cnt\hour_09_amt_cnt.joblib | MAE(AMT)=1,187,586.5 MAE(CNT)=21.96
[HOUR 10] saved -> ./models_hourly_amt_cnt\hour_10_amt_cnt.joblib | M